# Analyze number, size, and shape of stomata using interactive PlantCV tools
In this notebook we analyze brightfield microscopy images of plant stomata. First, the stoma are detected automatically using a color threshold; however, for most images this is insufficient to remove noise and get an accurate stomata count, depending on the quality of the images. Next, interactive tools can be used to select only the stomata. Lastly, the morphology package provides analysis of shape, including aperture (area), length (height), and width. Data is then exported as a .csv for downstream analysis. Stomatal conductance is a function of both the number of stoma and their aperture. 


## Import libraries and define local functions

In [2]:
%matplotlib widget

from matplotlib import pyplot as plt
from plantcv import plantcv as pcv
import cv2
import numpy as np
import argparse 
from  matplotlib import pyplot as plt
import os
from skimage import exposure, img_as_float
import glob
import pandas as pd

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
# Input/output options
class options:
    def __init__(self):
        self.image = "./imgs/SCALE_20x_1201_C_WT_4_001.tif"  # required
        self.debug = "plot"  # None, "print" (save to file), or "plot" (display in notebook)
        self.writeimg= False
        self.result = "example_results_oneimage_file.csv"  # required
        self.outdir = "." # Store the output to the current directory 

#Images in the format of .tif or .jpg are usable in this workflow. 
#We recommend keeping a spreadsheet with your image names so you can copy and paste them into the notebook one image at a time. 

# Get options
args = options()

In [4]:
# Set debug to the global parameter 
pcv.params.debug = args.debug
# Change display settings
pcv.params.dpi = 120
pcv.params.text_size = 10

NameError: name 'pcv' is not defined

## Load image and threshold 
The channel used and the threshold value can be adjusted for a better result. The resulting binary image from this step must contain all the stomata. The additional noise from other cells and shadows in the image will be removed in a subsequent step. 

In [ ]:
img, _, filename = pcv.readimage(filename=args.image, mode="native")
h,w,_ = img.shape

pcv.plot_image(img)
plt.title('RGB')

img_l = pcv.rgb2gray_lab(rgb_img=img, channel='l')

# Threshold the l channel image to get only the stomata
#If you don't like this threshold, you can try the other channels (a, l, v, s)
l_thresh = pcv.threshold.binary(gray_img=img_l, threshold=135, max_value=255, 
                                object_type='dark')

# post-processing for cleaning to fill in the center of the stomata
cross_kernel = pcv.get_kernel(size=(10,10), shape="cross")
l_fill_image = pcv.closing(gray_img=l_thresh, kernel = cross_kernel)

# look at the cleaned image and see if you have all the stomata, fully filled in.
pcv.plot_image(l_fill_image)
plt.title('Binary image containing all the stomata')

## Initialize interactive tool and use pre-detected stoma
Click to select only the stomata. You will need to click only once per stomata, and only inside or close to the dark center of the stoma. Your clicking does not have to be perfectly in the center of the stomata. If you click incorrectly, you can click again on the incorrect point to remove it. Adjust figsize for zoom as needed using the interactive figure. 

In [ ]:
#collect points for center of ROIs by clicking on each stomata. You do not need to click perfectly in them, just in or close to the dark center of the stomata
marker = pcv.Points(img, figsize=(8,8))

## Look at how many stomata you clicked in your image. You can analyze the number of stomata in your output .csv and do not need to record this unless you want to look at it sooner. 

In [ ]:
# This function measures the length of the list of marker points
len(marker.points)

## Create Regions of Interest (ROIs) around each of the markers you clicked, this will select only the stomata to move forward

In [ ]:
# Create ROI based on the selected points
# Your ROI will show up as blue circles here. 
roi_objects = pcv.roi.multi(img=img, coord=marker.points, radius=5)

## Identify all objects in the image. In the next step, you will only keep the objects in your ROIs. 

In [ ]:
# Identify objects
# Your objects will show up as pink here. 

# Inputs: 
#   img - RGB or grayscale image data for plotting 
#   mask - Binary mask used for detecting contours 
id_objects, obj_hierarchy = pcv.find_objects(img=img, mask=l_fill_image)

## Run a loop to measure each stomata individually and save each measurement out in a csv file for every individual stomata. 

In [ ]:
# Decide which objects to keep

# Inputs:
#    img            = img to display kept objects
#    roi_contour    = contour of roi, output from any ROI function
#    roi_hierarchy  = contour of roi, output from any ROI function
#    object_contour = contours of objects, output from pcv.find_objects function
#    obj_hierarchy  = hierarchy of objects, output from pcv.find_objects function
#    roi_type       = 'partial' (default, for partially inside the ROI), 'cutto', or 
#                     'largest' (keep only largest contour)

img_copy = np.copy(img)
pcv.params.debug = "none"
pcv.params.line_thickness = 1
i = 0

#make a mask to start with that has no stomata
mask_all_stomata = np.zeros(l_fill_image.shape, dtype=np.uint8)

for roi, hierarchy in roi_objects:
    # Find objects in the ROIs
    filtered_contours, filtered_hierarchy, filtered_mask, filtered_area = pcv.roi_objects(
        img=img, roi_type="partial", roi_contour=roi, roi_hierarchy=hierarchy, object_contour=id_objects, 
        obj_hierarchy=obj_hierarchy)
    
    mask_all_stomata = mask_all_stomata + filtered_mask
    
    # Combine objects together in each plant     
    plant_contour, plant_mask = pcv.object_composition(img=img_copy, contours=filtered_contours, hierarchy=filtered_hierarchy)        
    
    # Analyze the shape of each plant ----- You should change the text label to be specific for your issue, so that when you combine the csv for all images the stoma are labeled by the genotype, replicate, etc.-------------
    img_copy = pcv.analyze_object(img=img_copy, obj=plant_contour, mask=plant_mask, label=f"stoma_number_{i}")
    i = i + 1
    


## Calibration: Your output csv will be in units of pixels. In order to convert from pixels to other units, measure your scale bar. You should copy these values down to use in your downstream analysis, your results file will still show measurements in pixels. If you want to remain in pixels you can skip this step. 

In [ ]:
# Collect points using your mouse click for the left and right edges of your scale bar. 
scale_bar = pcv.Points(img, figsize=(8,8))

In [ ]:
# Use the x coordinate of the two points to get the equivalence between pixels and the image units.
# get x coordinates
x0 = scale_bar.points[0][0]
x1 = scale_bar.points[1][0]
print(f"The x coordinate of the two extremes of the scale bar are {x0} and {x1} respectively")

# calculate pixel equivalence 
scale = 100 # change this number to equal the micrometers in the image
bar_len_pix = x1-x0
print(f"The length in pixels of the scale bar is {bar_len_pix}")
one_pixel_in_units = scale/bar_len_pix
print(f"1 pixel is equivalent to {one_pixel_in_units} units")
print(f"Your .csv file will have data in pixel units, you can convert using this information when you analyze the numerical data")

## Save the results file and any image files you would like. "Plot" shows you the image here, "Print" saves the image to your designated folder. If you save out the masks, you can use these later for building deep learning pipelines of annotated images. 

In [ ]:
# Save the outputs as a csv
pcv.outputs.save_results(filename=args.result, outformat = "csv")

#plot the image with the analysis overlay so that you can see what it looks like and make sure the stomata are all included and that the measurement outline is around the stoma
pcv.plot_image(img_copy)

#save the mask image file if you would like to use it later for building deep learning applications
pcv.print_image(mask_all_stomata, filename = os.path.join(args.outdir, filename[:-4] + '_mask.png' ))

## Clear the outputs before you start on a new image so that each image has its own .csv file! 

In [ ]:
#clear outputs
pcv.outputs.clear()

## After you have done multiple images, combine the .csv files from each image into a single file for analysis

In [ ]:
# define the path to the individual result .csv files
path = 'args.outdir'
os.chdir(path)

#select which files should be combined
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

#combine the individual .csv files
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv.to_csv( "combined_results.csv", index=False, encoding='utf-8-sig')